This notebook is for normalising each TIR band separately then stacking them together into jpeg image chips. 

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os,sys
import matplotlib.pyplot as plt
from rasterio.plot import show, show_hist # some specific rasterio functions we'll need
import rasterio as rio
from rasterio.plot import adjust_band,reshape_as_image,show_hist


In [37]:
path = r'C:\\Users\\Maral.Bayaraa\\OneDrive - Satellite Applications Catapult\\Projects-Maral-Z\\SustainableFinance\\ALD_PhaseII\\GEE_imagechips_ML\\1_GEE_imagechips_tifs\\2_v3_dataset\\'
#outpath = r'C:\\Users\\Maral.Bayaraa\\OneDrive - Satellite Applications Catapult\\Projects-Maral-Z\\SustainableFinance\\ALD_Phase2\\GEE_imagechips_ML\\2_GEE_imagechips_jpeg\\v2_landcover_b10_RGB_201801_201701_201704'
#outpath = r'C:\\Users\\Maral.Bayaraa\\OneDrive - Satellite Applications Catapult\\Projects-Maral-Z\\SustainableFinance\\ALD_Phase2\\GEE_imagechips_ML\\3_GEE_imagechips_jpeg_datasetv3\\v3_cem_b10_RGB_201801_201701_201704\\'
outpath = r'C:\\Users\\Maral.Bayaraa\\OneDrive - Satellite Applications Catapult\\Projects-Maral-Z\\SustainableFinance\\ALD_Phase2\\GEE_imagechips_ML\\3_GEE_imagechips_jpeg_datasetv3\\v3_stl_b10_RGB_201801_201701_201704\\'


In [38]:
def normalize(x, lower, upper):
    """Stretch the max value to nan max and min to nan min"""

    x_max = np.nanmax(x)
    x_min = np.nanmin(x)

    m = (upper - lower) / (x_max - x_min)
    x_norm = (m * (x - x_min)) + lower

    return x_norm

In [39]:
def crop_center(img,cropx,cropy):
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

In [40]:
def combine_bands_saveasjpg(myinput):
    
    '''Please enter 
    "cem" for cement
    "lcv" for landcover
    "stl" for steel
    '''

    b1 = myinput+'_b10_201801'
    b2 = myinput+'_b10_201701'
    b3 = myinput+'_b10_201704'


    # cement
    path201801 = os.path.join(path,b1)

    path201701 = os.path.join(path,b2)
    path201704 = os.path.join(path,b3)


    print(b1)
    bands_201801 = os.listdir(path201801)
    bands_201704 = os.listdir(path201704)
    bands_201701 = os.listdir(path201701)



    jpegname = myinput
    #naming index to extract from name string
    num = 14
        
    
    for count,item in enumerate(bands_201801[:5]):
    
        band1 = item
        band2 = b1+str(item[num:])
        band3 = b2 +str(item[num:])
        band4 = b3+str(item[num:])
        
        '''****bug in naming convention of landcover for band4
        so when doing lc, change band4 to below:
        '''
        #band4 = b3+str(item[(num+1):])
            


        band1and2=[]
        rasters = []
        
        try:
            
            band1and2.append(band1)
            #print(os.path.join(path201801,band1))
            with rio.open(os.path.join(path201801,band1)) as f:
                rasters.append(f.read(1))
        except:
            print('failed:',band1)

            #see if the second band is in there

        
            #********* band3
        try:
            #print('opening 201701:', band3)
            #'''
            with rio.open(os.path.join(path201701,band3)) as f:
                rasters.append(f.read(1))
            #'''
            band1and2.append(band3)   
        except:
            print('failed:',band3)


        try:
            #********* band4
            #print('opening 201704:', band4)
            #
            with rio.open(os.path.join(path201704,band4)) as f:
                rasters.append(f.read(1))
            #
            band1and2.append(band4)       

        except:
            print('failed:',band4)
            continue

        
        
        
        # convert the list of files to a numpy array
        img = np.array(rasters, dtype=rasters[0].dtype)
        
        print('shape is',img.shape)

        #crop from centre to enfoce 
        data_crop = np.array([crop_center(img[i,:,:],35,35) for i in range(img.shape[0])])

        # Normalize each band separately
        data_norm = np.array([normalize(data_crop[i,:,:], 0, 255) for i in range(data_crop.shape[0])])

        #'''
        data_rgb = data_norm.astype("uint32")
        #print(data_rgb.shape)
        
        try:
            #changing bands and row col order
            data_rgb_reshpd = reshape_as_image(data_rgb)
            print('shape is',data_rgb_reshpd.shape)
            
            #ignoring NaN strips of data made up of small lines of pixels instead of matrices
            if data_rgb_reshpd.shape[0] > 20:

                #only runs if the try block was successfull
                plt.imshow(data_rgb_reshpd)
                plt.axis('off')
                #save figure without white cells around it
                plt.savefig(os.path.join(outpath,f'{jpegname}_{item[num:-4]}.jpg'),dpi=85,
                           bbox_inches='tight',
                           transparent=True,
                           pad_inches=0)
                plt.close()
                print(f'successfully saved: {jpegname}_{item[num:-4]}.jpg')
                
            else:
                print(f'******************failed to save {jpegname}_{item[num:-4]}.jpg******************************')
            
        #'''
        except:
            print(f'******************failed to save {jpegname}_{item[num:-4]}.jpg******************************')
            
             

# RESULTS

In [29]:
combine_bands_saveasjpg_nonorm('cem')

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


cem_b10_201801
shape is (3, 39, 47)
shape is (35, 35, 3)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


successfully saved: cem__1000.jpg
shape is (3, 39, 47)
shape is (35, 35, 3)
successfully saved: cem__1001.jpg
shape is (3, 40, 48)
shape is (35, 35, 3)
successfully saved: cem__1002.jpg


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

shape is (3, 40, 48)
shape is (35, 35, 3)
successfully saved: cem__992.jpg
shape is (3, 39, 47)
shape is (35, 35, 3)
successfully saved: cem__993.jpg
shape is (3, 39, 48)
shape is (35, 35, 3)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


successfully saved: cem__994.jpg
shape is (3, 39, 48)
shape is (35, 35, 3)
successfully saved: cem__995.jpg
shape is (3, 39, 48)
shape is (35, 35, 3)
successfully saved: cem__996.jpg


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


shape is (3, 39, 47)
shape is (35, 35, 3)
successfully saved: cem__997.jpg
shape is (3, 40, 48)
shape is (35, 35, 3)
successfully saved: cem__998.jpg
shape is (3, 39, 48)
shape is (35, 35, 3)
successfully saved: cem__999.jpg


In [31]:
combine_bands_saveasjpg_nonorm('stl')

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


stl_b10_201801
shape is (3, 36, 47)
shape is (35, 35, 3)
successfully saved: stl__175.jpg
shape is

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


 (3, 36, 48)
shape is (35, 35, 3)
successfully saved: stl__176.jpg
shape is (3, 36, 48)
shape is (35, 35, 3)
successfully saved: stl__177.jpg
shape is (3, 33, 48)
shape is (1, 35, 3)
******************failed to save stl__178.jpg******************************
shape is (3, 36, 47)
shape is (35, 35, 3)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

 (3, 40, 48)
shape is (35, 35, 3)
successfully saved: stl__282.jpg
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: stl__283.jpg
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: stl__284.jpg


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


shape is (3, 41, 47)
shape is (35, 35, 3)
successfully saved: stl__285.jpg
shape is (3, 41, 47)
shape is (35, 35, 3)
successfully saved: stl__286.jpg
shape is (3, 40, 48)
shape is (35, 35, 3)
successfully saved: stl__287.jpg


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


shape is (3, 40, 47)
shape is (35, 35, 3)
successfully saved: stl__288.jpg
shape is (3, 45, 48)
shape is (35, 35, 3)
successfully saved: stl__289.jpg
shape is (3, 46, 48)
shape is (35, 35, 3)
successfully saved: stl__290.jpg


In [8]:
## Only first 500 sites for landcover: 

combine_bands_saveasjpg_nonorm('lcv')

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


lcv_b10_201801
failed: lcv_b10_201701_0.tif
failed: lcv_b10_201704_0.tif
shape is (3, 46, 48)
shape is (35, 35, 3)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


successfully saved: lcv__1.jpg
failed: lcv_b10_201701_10.tif
shape is (2, 45, 48)
shape is (35, 35, 2)
******************failed to save lcv__10.jpg******************************
shape is (3, 44, 47)
shape is (35, 35, 3)
successfully saved: lcv__100.jpg
shape is (3, 41, 47)
shape is (35, 35, 3)
successfully saved: lcv__1000.jpg


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

shape is (3, 39, 48)
shape is (35, 35, 3)
successfully saved: lcv__1442.jpg
shape is (3, 39, 47)
shape is (35, 35, 3)
successfully saved: lcv__1443.jpg


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


shape is (3, 38, 47)
shape is (35, 35, 3)
successfully saved: lcv__1444.jpg
shape is (3, 38, 48)
shape is (35, 35, 3)
successfully saved: lcv__1445.jpg
shape is (3, 39, 48)
shape is (35, 35, 3)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


successfully saved: lcv__1446.jpg
shape is (3, 39, 47)
shape is (35, 35, 3)
successfully saved: lcv__1447.jpg


# Three band output

combine_bands_saveasjpg('cem')

In [155]:
combine_bands_saveasjpg('stl')

stl_b10_201801
shape is (3, 36, 47)
shape is (35, 35, 3)
successfully saved: stl__175.jpg
shape is (3, 36, 48)
shape is (35, 35, 3)
successfully saved: stl__176.jpg
shape is (3, 36, 48)
shape is (35, 35, 3)
successfully saved: stl__177.jpg
shape is (3, 33, 48)
shape is (1, 35, 3)
******************failed to save stl__178.jpg******************************
shape is (3, 36, 47)
shape is (35, 35, 3)
successfully saved: stl__179.jpg
shape is (3, 36, 48)
shape is (35, 35, 3)
successfully saved: stl__180.jpg
shape is (3, 36, 47)
shape is (35, 35, 3)
successfully saved: stl__181.jpg
shape is (3, 36, 48)
shape is (35, 35, 3)
successfully saved: stl__182.jpg
shape is (3, 35, 48)
shape is (35, 35, 3)
successfully saved: stl__183.jpg
shape is (3, 36, 48)
shape is (35, 35, 3)
successfully saved: stl__184.jpg
shape is (3, 36, 48)
shape is (35, 35, 3)
successfully saved: stl__185.jpg
shape is (3, 35, 47)
shape is (35, 35, 3)
successfully saved: stl__186.jpg
shape is (3, 35, 48)
shape is (35, 35, 3)
s

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: All-NaN slice encountered
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: All-NaN slice encountered
  """


successfully saved: stl__205.jpg
shape is (3, 37, 48)
shape is (35, 35, 3)
successfully saved: stl__206.jpg
shape is (3, 37, 48)
shape is (35, 35, 3)
successfully saved: stl__207.jpg
shape is (3, 37, 48)
shape is (35, 35, 3)
successfully saved: stl__208.jpg
shape is (3, 37, 47)
shape is (35, 35, 3)
successfully saved: stl__209.jpg
shape is (3, 37, 48)
shape is (35, 35, 3)
successfully saved: stl__210.jpg
shape is (3, 37, 47)
shape is (35, 35, 3)
successfully saved: stl__211.jpg
shape is (3, 37, 48)
shape is (35, 35, 3)
successfully saved: stl__212.jpg
shape is (3, 38, 48)
shape is (35, 35, 3)
successfully saved: stl__213.jpg
shape is (3, 38, 47)
shape is (35, 35, 3)
successfully saved: stl__214.jpg
shape is (3, 42, 48)
shape is (35, 35, 3)
successfully saved: stl__215.jpg
shape is (3, 38, 48)
shape is (35, 35, 3)
successfully saved: stl__216.jpg
shape is (3, 43, 47)
shape is (35, 35, 3)
successfully saved: stl__217.jpg
shape is (3, 42, 48)
shape is (35, 35, 3)
successfully saved: stl__

In [163]:
combine_bands_saveasjpg('lcv')

lcv_b10_201801
failed: lcv_b10_201701_0.tif
failed: lcv_b10_201704_0.tif
shape is (3, 46, 48)
shape is (35, 35, 3)
successfully saved: lcv__1.jpg
failed: lcv_b10_201701_10.tif
shape is (2, 45, 48)
shape is (35, 35, 2)
******************failed to save lcv__10.jpg******************************
shape is (3, 44, 47)
shape is (35, 35, 3)
successfully saved: lcv__100.jpg
shape is (3, 41, 47)
shape is (35, 35, 3)
successfully saved: lcv__1000.jpg
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__1001.jpg
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__1002.jpg
shape is (3, 41, 47)
shape is (35, 35, 3)
successfully saved: lcv__1003.jpg
shape is (3, 41, 47)
shape is (35, 35, 3)
successfully saved: lcv__1004.jpg
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__1005.jpg
failed: lcv_b10_201701_1006.tif
failed: lcv_b10_201704_1006.tif
shape is (3, 42, 47)
shape is (35, 35, 3)
successfully saved: lcv__1007.jpg
failed: lcv_b10_201701_1008.tif
shap

successfully saved: lcv__1079.jpg
failed: lcv_b10_201701_108.tif
shape is (2, 44, 48)
shape is (35, 35, 2)
******************failed to save lcv__108.jpg******************************
failed: lcv_b10_201701_1080.tif
shape is (2, 41, 48)
shape is (35, 35, 2)
******************failed to save lcv__1080.jpg******************************
shape is (3, 41, 47)
shape is (35, 35, 3)
successfully saved: lcv__1081.jpg
failed: lcv_b10_201701_1082.tif
shape is (2, 41, 47)
shape is (35, 35, 2)
******************failed to save lcv__1082.jpg******************************
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__1083.jpg
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__1084.jpg
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__1085.jpg
failed: lcv_b10_201701_1086.tif
failed: lcv_b10_201704_1086.tif
shape is (3, 41, 47)
shape is (35, 35, 3)
successfully saved: lcv__1087.jpg
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__1088

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: All-NaN slice encountered
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: All-NaN slice encountered
  """


successfully saved: lcv__1120.jpg
failed: lcv_b10_201704_1121.tif
failed: lcv_b10_201701_1122.tif
failed: lcv_b10_201704_1122.tif
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__1123.jpg
failed: lcv_b10_201701_1124.tif
shape is (2, 41, 48)
shape is (35, 35, 2)
******************failed to save lcv__1124.jpg******************************
failed: lcv_b10_201701_1125.tif
shape is (2, 41, 48)
shape is (35, 35, 2)
******************failed to save lcv__1125.jpg******************************
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__1126.jpg
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__1127.jpg
failed: lcv_b10_201701_1128.tif
shape is (2, 41, 48)
shape is (35, 35, 2)
******************failed to save lcv__1128.jpg******************************
shape is (3, 40, 48)
shape is (35, 35, 3)
successfully saved: lcv__1129.jpg
shape is (3, 44, 48)
shape is (35, 35, 3)
successfully saved: lcv__113.jpg
shape is (3, 40, 48)
shape is (35, 35,

shape is (3, 45, 48)
shape is (35, 35, 3)
successfully saved: lcv__12.jpg
shape is (3, 44, 48)
shape is (35, 35, 3)
successfully saved: lcv__120.jpg
failed: lcv_b10_201701_1200.tif
shape is (2, 39, 48)
shape is (35, 35, 2)
******************failed to save lcv__1200.jpg******************************
shape is (3, 39, 47)
shape is (35, 35, 3)
successfully saved: lcv__1201.jpg
shape is (3, 40, 47)
shape is (35, 35, 3)
successfully saved: lcv__1202.jpg
failed: lcv_b10_201701_1203.tif
failed: lcv_b10_201704_1203.tif
shape is (3, 40, 48)
shape is (35, 35, 3)
successfully saved: lcv__1204.jpg
shape is (3, 40, 48)
shape is (35, 35, 3)
successfully saved: lcv__1205.jpg
shape is (3, 40, 48)
shape is (35, 35, 3)
successfully saved: lcv__1206.jpg
shape is (3, 40, 48)
shape is (35, 35, 3)
successfully saved: lcv__1207.jpg
shape is (3, 40, 48)
shape is (35, 35, 3)
successfully saved: lcv__1208.jpg
shape is (3, 40, 47)
shape is (35, 35, 3)
successfully saved: lcv__1209.jpg
shape is (3, 44, 47)
shape i

...

shape is (3, 44, 48)
shape is (35, 35, 3)
successfully saved: lcv__68.jpg
failed: lcv_b10_201701_680.tif
failed: lcv_b10_201704_680.tif
shape is (3, 42, 47)
shape is (35, 35, 3)
successfully saved: lcv__681.jpg
shape is (3, 42, 47)
shape is (35, 35, 3)
successfully saved: lcv__682.jpg
failed: lcv_b10_201701_683.tif
failed: lcv_b10_201704_683.tif
shape is (3, 41, 47)
shape is (35, 35, 3)
successfully saved: lcv__684.jpg
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__685.jpg
failed: lcv_b10_201701_686.tif
shape is (2, 42, 48)
shape is (35, 35, 2)
******************failed to save lcv__686.jpg******************************
shape is (3, 42, 48)
shape is (35, 35, 3)
successfully saved: lcv__687.jpg
shape is (3, 42, 48)
shape is (35, 35, 3)
successfully saved: lcv__688.jpg
shape is (3, 41, 47)
shape is (35, 35, 3)
successfully saved: lcv__689.jpg
failed: lcv_b10_201701_69.tif
shape is (2, 44, 47)
shape is (35, 35, 2)
******************failed to save lcv__69.jpg************

shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__756.jpg
shape is (3, 42, 48)
shape is (35, 35, 3)
successfully saved: lcv__757.jpg
shape is (3, 42, 48)
shape is (35, 35, 3)
successfully saved: lcv__758.jpg
failed: lcv_b10_201701_759.tif
shape is (2, 41, 48)
shape is (35, 35, 2)
******************failed to save lcv__759.jpg******************************
shape is (3, 44, 48)
shape is (35, 35, 3)
successfully saved: lcv__76.jpg
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__760.jpg
failed: lcv_b10_201701_761.tif
shape is (2, 41, 48)
shape is (35, 35, 2)
******************failed to save lcv__761.jpg******************************
failed: lcv_b10_201701_762.tif
shape is (2, 41, 48)
shape is (35, 35, 2)
******************failed to save lcv__762.jpg******************************
failed: lcv_b10_201701_763.tif
shape is (2, 41, 48)
shape is (35, 35, 2)
******************failed to save lcv__763.jpg******************************
shape is (3, 41, 48)
shape is 

shape is (3, 42, 48)
shape is (35, 35, 3)
successfully saved: lcv__839.jpg
failed: lcv_b10_201701_84.tif
shape is (2, 44, 48)
shape is (35, 35, 2)
******************failed to save lcv__84.jpg******************************
failed: lcv_b10_201701_840.tif
shape is (2, 42, 48)
shape is (35, 35, 2)
******************failed to save lcv__840.jpg******************************
failed: lcv_b10_201701_841.tif
shape is (2, 41, 48)
shape is (35, 35, 2)
******************failed to save lcv__841.jpg******************************
failed: lcv_b10_201701_842.tif
failed: lcv_b10_201704_842.tif
shape is (3, 42, 48)
shape is (35, 35, 3)
successfully saved: lcv__843.jpg
failed: lcv_b10_201701_844.tif
shape is (2, 41, 48)
shape is (35, 35, 2)
******************failed to save lcv__844.jpg******************************
failed: lcv_b10_201701_845.tif
shape is (2, 41, 48)
shape is (35, 35, 2)
******************failed to save lcv__845.jpg******************************
shape is (3, 41, 48)
shape is (35, 35, 3)
suc

shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__914.jpg
shape is (3, 42, 48)
shape is (35, 35, 3)
successfully saved: lcv__915.jpg
shape is (3, 42, 47)
shape is (35, 35, 3)
successfully saved: lcv__916.jpg
shape is (3, 41, 47)
shape is (35, 35, 3)
successfully saved: lcv__917.jpg
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__918.jpg
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__919.jpg
shape is (3, 44, 47)
shape is (35, 35, 3)
successfully saved: lcv__92.jpg
failed: lcv_b10_201701_920.tif
shape is (2, 42, 48)
shape is (35, 35, 2)
******************failed to save lcv__920.jpg******************************
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__921.jpg
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__922.jpg
shape is (3, 42, 48)
shape is (35, 35, 3)
successfully saved: lcv__923.jpg
failed: lcv_b10_201701_924.tif
shape is (2, 42, 48)
shape is (35, 35, 2)
******************failed to s

failed: lcv_b10_201701_989.tif
failed: lcv_b10_201704_989.tif
failed: lcv_b10_201701_99.tif
failed: lcv_b10_201704_99.tif
shape is (3, 41, 47)
shape is (35, 35, 3)
successfully saved: lcv__990.jpg
failed: lcv_b10_201701_991.tif
shape is (2, 41, 48)
shape is (35, 35, 2)
******************failed to save lcv__991.jpg******************************
failed: lcv_b10_201701_992.tif
shape is (2, 42, 48)
shape is (35, 35, 2)
******************failed to save lcv__992.jpg******************************
failed: lcv_b10_201701_993.tif
failed: lcv_b10_201704_993.tif
shape is (3, 41, 48)
shape is (35, 35, 3)
successfully saved: lcv__994.jpg
shape is (3, 41, 47)
shape is (35, 35, 3)
successfully saved: lcv__995.jpg
failed: lcv_b10_201701_996.tif
shape is (2, 41, 47)
shape is (35, 35, 2)
******************failed to save lcv__996.jpg******************************
failed: lcv_b10_201701_997.tif
shape is (2, 41, 48)
shape is (35, 35, 2)
******************failed to save lcv__997.jpg**************************

In [21]:
combine_bands_saveasjpg('cem')

cem_b10_201801
successfully saved: cem__1000.jpg
successfully saved: cem__1001.jpg
successfully saved: cem__1002.jpg
successfully saved: cem__1003.jpg
successfully saved: cem__1004.jpg
successfully saved: cem__1005.jpg
successfully saved: cem__1006.jpg
successfully saved: cem__1007.jpg
successfully saved: cem__1008.jpg
successfully saved: cem__1009.jpg
successfully saved: cem__1010.jpg
successfully saved: cem__1011.jpg
successfully saved: cem__1012.jpg
successfully saved: cem__1013.jpg
successfully saved: cem__1014.jpg
successfully saved: cem__1015.jpg
successfully saved: cem__1016.jpg
successfully saved: cem__1017.jpg
successfully saved: cem__1019.jpg
successfully saved: cem__1020.jpg
successfully saved: cem__1021.jpg
successfully saved: cem__1022.jpg
successfully saved: cem__1023.jpg
successfully saved: cem__1024.jpg
successfully saved: cem__1025.jpg
successfully saved: cem__1026.jpg
successfully saved: cem__1027.jpg
successfully saved: cem__1028.jpg
successfully saved: cem__1029.jpg

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: All-NaN slice encountered
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: All-NaN slice encountered
  """


successfully saved: cem__329.jpg
successfully saved: cem__334.jpg
successfully saved: cem__336.jpg
successfully saved: cem__344.jpg
successfully saved: cem__347.jpg
successfully saved: cem__348.jpg
successfully saved: cem__350.jpg
successfully saved: cem__353.jpg
successfully saved: cem__357.jpg
successfully saved: cem__358.jpg
successfully saved: cem__363.jpg
successfully saved: cem__366.jpg
successfully saved: cem__369.jpg
successfully saved: cem__370.jpg
successfully saved: cem__376.jpg
successfully saved: cem__378.jpg
successfully saved: cem__379.jpg
successfully saved: cem__380.jpg
successfully saved: cem__381.jpg
successfully saved: cem__382.jpg
successfully saved: cem__383.jpg
successfully saved: cem__385.jpg
successfully saved: cem__386.jpg
successfully saved: cem__387.jpg
successfully saved: cem__398.jpg
successfully saved: cem__411.jpg
successfully saved: cem__414.jpg
successfully saved: cem__417.jpg
successfully saved: cem__418.jpg
successfully saved: cem__421.jpg
successful

successfully saved: cem__935.jpg
successfully saved: cem__936.jpg
successfully saved: cem__937.jpg
successfully saved: cem__938.jpg
successfully saved: cem__939.jpg
successfully saved: cem__940.jpg
successfully saved: cem__941.jpg
successfully saved: cem__942.jpg
successfully saved: cem__943.jpg
successfully saved: cem__944.jpg
successfully saved: cem__945.jpg
successfully saved: cem__946.jpg
successfully saved: cem__947.jpg
successfully saved: cem__948.jpg
successfully saved: cem__949.jpg
successfully saved: cem__950.jpg
successfully saved: cem__951.jpg
successfully saved: cem__952.jpg
successfully saved: cem__953.jpg
successfully saved: cem__954.jpg
successfully saved: cem__955.jpg
successfully saved: cem__956.jpg
successfully saved: cem__957.jpg
successfully saved: cem__958.jpg
successfully saved: cem__959.jpg
successfully saved: cem__960.jpg
successfully saved: cem__961.jpg
successfully saved: cem__962.jpg
successfully saved: cem__963.jpg
successfully saved: cem__964.jpg
successful

In [26]:
combine_bands_saveasjpg('stl')

stl_b10_201801
successfully saved: stl__175.jpg
successfully saved: stl__176.jpg
successfully saved: stl__177.jpg
successfully saved: stl__178.jpg
successfully saved: stl__179.jpg
successfully saved: stl__180.jpg
successfully saved: stl__181.jpg
successfully saved: stl__182.jpg
successfully saved: stl__183.jpg
successfully saved: stl__184.jpg
successfully saved: stl__185.jpg
successfully saved: stl__186.jpg
successfully saved: stl__187.jpg
successfully saved: stl__188.jpg
successfully saved: stl__189.jpg
successfully saved: stl__190.jpg
successfully saved: stl__191.jpg
successfully saved: stl__192.jpg
successfully saved: stl__193.jpg
successfully saved: stl__194.jpg
successfully saved: stl__195.jpg
successfully saved: stl__196.jpg
successfully saved: stl__197.jpg
successfully saved: stl__198.jpg
successfully saved: stl__199.jpg
successfully saved: stl__200.jpg
successfully saved: stl__201.jpg
successfully saved: stl__202.jpg


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: All-NaN slice encountered
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: All-NaN slice encountered
  """


successfully saved: stl__203.jpg
successfully saved: stl__204.jpg
successfully saved: stl__205.jpg
successfully saved: stl__206.jpg
successfully saved: stl__207.jpg
successfully saved: stl__208.jpg
successfully saved: stl__209.jpg
successfully saved: stl__210.jpg
successfully saved: stl__211.jpg
successfully saved: stl__212.jpg
successfully saved: stl__213.jpg
successfully saved: stl__214.jpg
successfully saved: stl__215.jpg
successfully saved: stl__216.jpg
successfully saved: stl__217.jpg
successfully saved: stl__218.jpg
successfully saved: stl__219.jpg
successfully saved: stl__220.jpg
successfully saved: stl__221.jpg
successfully saved: stl__222.jpg
successfully saved: stl__223.jpg
successfully saved: stl__224.jpg
successfully saved: stl__225.jpg
successfully saved: stl__226.jpg
successfully saved: stl__227.jpg
successfully saved: stl__228.jpg
successfully saved: stl__229.jpg
successfully saved: stl__230.jpg
successfully saved: stl__231.jpg
successfully saved: stl__232.jpg
successful

In [28]:
combine_bands_saveasjpg('lcv')

lcv_b10_201801
failed: lcv_b10_201701_0.tif
failed: lcv_b10_201704_0.tif
successfully saved: lcv__1.jpg
failed: lcv_b10_201701_10.tif
******************failed to save lcv__10.jpg******************************
successfully saved: lcv__100.jpg
successfully saved: lcv__1000.jpg
successfully saved: lcv__1001.jpg
successfully saved: lcv__1002.jpg
successfully saved: lcv__1003.jpg
successfully saved: lcv__1004.jpg
successfully saved: lcv__1005.jpg
failed: lcv_b10_201701_1006.tif
failed: lcv_b10_201704_1006.tif
successfully saved: lcv__1007.jpg
failed: lcv_b10_201701_1008.tif
******************failed to save lcv__1008.jpg******************************
successfully saved: lcv__1009.jpg
successfully saved: lcv__101.jpg
failed: lcv_b10_201701_1010.tif
failed: lcv_b10_201704_1010.tif
successfully saved: lcv__1011.jpg
successfully saved: lcv__1012.jpg
successfully saved: lcv__1013.jpg
failed: lcv_b10_201701_1014.tif
failed: lcv_b10_201704_1014.tif
successfully saved: lcv__1015.jpg
successfully sav

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: All-NaN slice encountered
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: All-NaN slice encountered
  """


successfully saved: lcv__112.jpg
successfully saved: lcv__1120.jpg
failed: lcv_b10_201704_1121.tif
failed: lcv_b10_201701_1122.tif
failed: lcv_b10_201704_1122.tif
successfully saved: lcv__1123.jpg
failed: lcv_b10_201701_1124.tif
******************failed to save lcv__1124.jpg******************************
failed: lcv_b10_201701_1125.tif
******************failed to save lcv__1125.jpg******************************
successfully saved: lcv__1126.jpg
successfully saved: lcv__1127.jpg
failed: lcv_b10_201701_1128.tif
******************failed to save lcv__1128.jpg******************************
successfully saved: lcv__1129.jpg
successfully saved: lcv__113.jpg
successfully saved: lcv__1130.jpg
failed: lcv_b10_201701_1131.tif
******************failed to save lcv__1131.jpg******************************
successfully saved: lcv__1132.jpg
successfully saved: lcv__1133.jpg
successfully saved: lcv__1134.jpg
failed: lcv_b10_201701_1135.tif
******************failed to save lcv__1135.jpg******************

successfully saved: lcv__1240.jpg
failed: lcv_b10_201701_1241.tif
failed: lcv_b10_201704_1241.tif
failed: lcv_b10_201701_1242.tif
******************failed to save lcv__1242.jpg******************************
successfully saved: lcv__1243.jpg
successfully saved: lcv__1244.jpg
failed: lcv_b10_201701_1245.tif
******************failed to save lcv__1245.jpg******************************
failed: lcv_b10_201701_1246.tif
******************failed to save lcv__1246.jpg******************************
successfully saved: lcv__1247.jpg
successfully saved: lcv__1248.jpg
failed: lcv_b10_201701_1249.tif
******************failed to save lcv__1249.jpg******************************
successfully saved: lcv__125.jpg
failed: lcv_b10_201701_1250.tif
******************failed to save lcv__1250.jpg******************************
successfully saved: lcv__1251.jpg
failed: lcv_b10_201701_1252.tif
******************failed to save lcv__1252.jpg******************************
failed: lcv_b10_201701_1253.tif
*************

successfully saved: lcv__1362.jpg
successfully saved: lcv__1363.jpg
failed: lcv_b10_201701_1364.tif
******************failed to save lcv__1364.jpg******************************
failed: lcv_b10_201701_1365.tif
******************failed to save lcv__1365.jpg******************************
successfully saved: lcv__1366.jpg
failed: lcv_b10_201701_1367.tif
failed: lcv_b10_201704_1367.tif
failed: lcv_b10_201701_1368.tif
failed: lcv_b10_201704_1368.tif
failed: lcv_b10_201701_1369.tif
******************failed to save lcv__1369.jpg******************************
successfully saved: lcv__137.jpg
successfully saved: lcv__1370.jpg
failed: lcv_b10_201701_1371.tif
******************failed to save lcv__1371.jpg******************************
failed: lcv_b10_201701_1372.tif
******************failed to save lcv__1372.jpg******************************
failed: lcv_b10_201701_1373.tif
******************failed to save lcv__1373.jpg******************************
successfully saved: lcv__1374.jpg
successfully sa

...

failed: lcv_b10_201701_472.tif
failed: lcv_b10_201704_472.tif
failed: lcv_b10_201701_473.tif
******************failed to save lcv__473.jpg******************************
failed: lcv_b10_201701_474.tif
******************failed to save lcv__474.jpg******************************
successfully saved: lcv__475.jpg
successfully saved: lcv__476.jpg
successfully saved: lcv__477.jpg
failed: lcv_b10_201701_478.tif
******************failed to save lcv__478.jpg******************************
successfully saved: lcv__479.jpg
successfully saved: lcv__48.jpg
failed: lcv_b10_201701_480.tif
******************failed to save lcv__480.jpg******************************
failed: lcv_b10_201701_481.tif
******************failed to save lcv__481.jpg******************************
failed: lcv_b10_201701_482.tif
******************failed to save lcv__482.jpg******************************
failed: lcv_b10_201701_483.tif
failed: lcv_b10_201704_483.tif
failed: lcv_b10_201701_484.tif
******************failed to save lcv__4

failed: lcv_b10_201701_598.tif
******************failed to save lcv__598.jpg******************************
successfully saved: lcv__599.jpg
failed: lcv_b10_201701_6.tif
******************failed to save lcv__6.jpg******************************
successfully saved: lcv__60.jpg
failed: lcv_b10_201701_600.tif
******************failed to save lcv__600.jpg******************************
failed: lcv_b10_201701_601.tif
******************failed to save lcv__601.jpg******************************
successfully saved: lcv__602.jpg
successfully saved: lcv__603.jpg
failed: lcv_b10_201701_604.tif
******************failed to save lcv__604.jpg******************************
failed: lcv_b10_201701_605.tif
******************failed to save lcv__605.jpg******************************
successfully saved: lcv__606.jpg
failed: lcv_b10_201701_607.tif
failed: lcv_b10_201704_607.tif
failed: lcv_b10_201701_608.tif
failed: lcv_b10_201704_608.tif
successfully saved: lcv__609.jpg
successfully saved: lcv__61.jpg
successfu

successfully saved: lcv__724.jpg
successfully saved: lcv__725.jpg
successfully saved: lcv__726.jpg
failed: lcv_b10_201701_727.tif
******************failed to save lcv__727.jpg******************************
successfully saved: lcv__728.jpg
successfully saved: lcv__729.jpg
failed: lcv_b10_201701_73.tif
******************failed to save lcv__73.jpg******************************
successfully saved: lcv__730.jpg
failed: lcv_b10_201701_731.tif
******************failed to save lcv__731.jpg******************************
successfully saved: lcv__732.jpg
successfully saved: lcv__733.jpg
failed: lcv_b10_201701_734.tif
******************failed to save lcv__734.jpg******************************
failed: lcv_b10_201701_735.tif
******************failed to save lcv__735.jpg******************************
successfully saved: lcv__736.jpg
successfully saved: lcv__737.jpg
failed: lcv_b10_201701_738.tif
failed: lcv_b10_201704_738.tif
successfully saved: lcv__739.jpg
successfully saved: lcv__74.jpg
successful

successfully saved: lcv__853.jpg
failed: lcv_b10_201701_854.tif
******************failed to save lcv__854.jpg******************************
successfully saved: lcv__855.jpg
failed: lcv_b10_201701_856.tif
******************failed to save lcv__856.jpg******************************
failed: lcv_b10_201704_857.tif
failed: lcv_b10_201701_858.tif
******************failed to save lcv__858.jpg******************************
failed: lcv_b10_201701_859.tif
failed: lcv_b10_201704_859.tif
successfully saved: lcv__86.jpg
failed: lcv_b10_201701_860.tif
******************failed to save lcv__860.jpg******************************
successfully saved: lcv__861.jpg
failed: lcv_b10_201701_862.tif
******************failed to save lcv__862.jpg******************************
successfully saved: lcv__863.jpg
failed: lcv_b10_201701_864.tif
******************failed to save lcv__864.jpg******************************
successfully saved: lcv__865.jpg
failed: lcv_b10_201701_866.tif
******************failed to save lcv_

successfully saved: lcv__974.jpg
failed: lcv_b10_201701_975.tif
failed: lcv_b10_201704_975.tif
successfully saved: lcv__976.jpg
failed: lcv_b10_201701_977.tif
******************failed to save lcv__977.jpg******************************
successfully saved: lcv__978.jpg
successfully saved: lcv__979.jpg
successfully saved: lcv__98.jpg
successfully saved: lcv__980.jpg
failed: lcv_b10_201704_981.tif
failed: lcv_b10_201701_982.tif
******************failed to save lcv__982.jpg******************************
successfully saved: lcv__983.jpg
failed: lcv_b10_201701_984.tif
******************failed to save lcv__984.jpg******************************
failed: lcv_b10_201701_985.tif
failed: lcv_b10_201704_985.tif
successfully saved: lcv__986.jpg
failed: lcv_b10_201701_987.tif
******************failed to save lcv__987.jpg******************************
successfully saved: lcv__988.jpg
failed: lcv_b10_201701_989.tif
failed: lcv_b10_201704_989.tif
failed: lcv_b10_201701_99.tif
failed: lcv_b10_201704_99.tif

### Old runs

For lc, make sure to change the band 4 config above (smth to do with a bug in naming convention )

In [115]:
combine_bands_saveasjpg('lc')

successfully saved: lc__0.jpg
successfully saved: lc__1.jpg
successfully saved: lc__10.jpg
successfully saved: lc__100.jpg
******************failed to save lc__1000.jpg******************************
******************failed to save lc__1001.jpg******************************
failed: Landcover_b10_201701_1002.tif
******************failed to save lc__1002.jpg******************************
******************failed to save lc__1003.jpg******************************
******************failed to save lc__1004.jpg******************************
******************failed to save lc__1005.jpg******************************
******************failed to save lc__1006.jpg******************************
******************failed to save lc__1007.jpg******************************
******************failed to save lc__1008.jpg******************************
successfully saved: lc__1009.jpg
successfully saved: lc__101.jpg
successfully saved: lc__1010.jpg
******************failed to save lc__1011.jpg************

******************failed to save lc__1100.jpg******************************
******************failed to save lc__1101.jpg******************************
******************failed to save lc__1102.jpg******************************
******************failed to save lc__1103.jpg******************************
******************failed to save lc__1104.jpg******************************
******************failed to save lc__1105.jpg******************************
******************failed to save lc__1106.jpg******************************
******************failed to save lc__1107.jpg******************************
******************failed to save lc__1108.jpg******************************
******************failed to save lc__1109.jpg******************************
successfully saved: lc__111.jpg


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: All-NaN slice encountered
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: All-NaN slice encountered
  """


failed: Landcover_b10_201701_1110.tif
******************failed to save lc__1110.jpg******************************
******************failed to save lc__1111.jpg******************************
******************failed to save lc__1112.jpg******************************
******************failed to save lc__1113.jpg******************************
******************failed to save lc__1114.jpg******************************
******************failed to save lc__1115.jpg******************************
******************failed to save lc__1116.jpg******************************
******************failed to save lc__1117.jpg******************************
******************failed to save lc__1118.jpg******************************
******************failed to save lc__1119.jpg******************************
successfully saved: lc__112.jpg
******************failed to save lc__1120.jpg******************************
******************failed to save lc__1121.jpg******************************
******************

successfully saved: lc__364.jpg
successfully saved: lc__365.jpg
successfully saved: lc__366.jpg
successfully saved: lc__367.jpg
successfully saved: lc__368.jpg
successfully saved: lc__369.jpg
failed: Landcover_b10_201701_37.tif
******************failed to save lc__37.jpg******************************
successfully saved: lc__370.jpg
successfully saved: lc__371.jpg
successfully saved: lc__372.jpg
successfully saved: lc__373.jpg
successfully saved: lc__374.jpg
successfully saved: lc__375.jpg
successfully saved: lc__376.jpg
successfully saved: lc__377.jpg
successfully saved: lc__378.jpg
successfully saved: lc__379.jpg
successfully saved: lc__38.jpg
successfully saved: lc__380.jpg
successfully saved: lc__381.jpg
successfully saved: lc__382.jpg
successfully saved: lc__383.jpg
successfully saved: lc__384.jpg
successfully saved: lc__385.jpg
successfully saved: lc__386.jpg
successfully saved: lc__387.jpg
successfully saved: lc__388.jpg
successfully saved: lc__389.jpg
successfully saved: lc__39.

successfully saved: lc__555.jpg
successfully saved: lc__556.jpg
successfully saved: lc__557.jpg
successfully saved: lc__558.jpg
successfully saved: lc__559.jpg
successfully saved: lc__56.jpg
successfully saved: lc__560.jpg
successfully saved: lc__561.jpg
successfully saved: lc__562.jpg
successfully saved: lc__563.jpg
successfully saved: lc__564.jpg
successfully saved: lc__565.jpg
successfully saved: lc__566.jpg
failed: Landcover_b10_201701_567.tif
******************failed to save lc__567.jpg******************************
successfully saved: lc__568.jpg
successfully saved: lc__569.jpg
successfully saved: lc__57.jpg
successfully saved: lc__570.jpg
successfully saved: lc__571.jpg
successfully saved: lc__572.jpg
successfully saved: lc__573.jpg
successfully saved: lc__574.jpg
successfully saved: lc__575.jpg
successfully saved: lc__576.jpg
successfully saved: lc__577.jpg
successfully saved: lc__578.jpg
successfully saved: lc__579.jpg
successfully saved: lc__58.jpg
successfully saved: lc__580

successfully saved: lc__76.jpg
successfully saved: lc__760.jpg
successfully saved: lc__761.jpg
failed: Landcover_b10_201701_762.tif
******************failed to save lc__762.jpg******************************
successfully saved: lc__763.jpg
successfully saved: lc__764.jpg
successfully saved: lc__765.jpg
failed: Landcover_b10_201701_766.tif
******************failed to save lc__766.jpg******************************
successfully saved: lc__767.jpg
successfully saved: lc__768.jpg
successfully saved: lc__769.jpg
successfully saved: lc__77.jpg
successfully saved: lc__770.jpg
successfully saved: lc__771.jpg
successfully saved: lc__772.jpg
successfully saved: lc__773.jpg
successfully saved: lc__774.jpg
successfully saved: lc__775.jpg
successfully saved: lc__776.jpg
successfully saved: lc__777.jpg
successfully saved: lc__778.jpg
successfully saved: lc__779.jpg
successfully saved: lc__78.jpg
successfully saved: lc__780.jpg
successfully saved: lc__781.jpg
successfully saved: lc__782.jpg
successfull

******************failed to save lc__959.jpg******************************
successfully saved: lc__96.jpg
successfully saved: lc__960.jpg
successfully saved: lc__961.jpg
successfully saved: lc__962.jpg
successfully saved: lc__963.jpg
successfully saved: lc__964.jpg
successfully saved: lc__965.jpg
successfully saved: lc__966.jpg
successfully saved: lc__967.jpg
successfully saved: lc__968.jpg
successfully saved: lc__969.jpg
successfully saved: lc__97.jpg
failed: Landcover_b10_201701_970.tif
******************failed to save lc__970.jpg******************************
******************failed to save lc__971.jpg******************************
successfully saved: lc__972.jpg
successfully saved: lc__973.jpg
******************failed to save lc__974.jpg******************************
******************failed to save lc__975.jpg******************************
successfully saved: lc__976.jpg
******************failed to save lc__977.jpg******************************
******************failed to save lc

In [122]:
combine_bands_saveasjpg('cement')

cement
successfully saved: cem__1000.jpg
successfully saved: cem__1001.jpg
successfully saved: cem__1002.jpg
successfully saved: cem__1003.jpg
successfully saved: cem__1004.jpg
successfully saved: cem__1005.jpg
successfully saved: cem__1006.jpg
successfully saved: cem__1007.jpg
successfully saved: cem__1008.jpg
successfully saved: cem__1009.jpg
successfully saved: cem__1010.jpg
successfully saved: cem__1011.jpg
successfully saved: cem__1012.jpg
successfully saved: cem__1013.jpg
successfully saved: cem__1014.jpg
successfully saved: cem__1015.jpg
successfully saved: cem__1016.jpg
******************failed to save cem__1017.jpg******************************
successfully saved: cem__1019.jpg
successfully saved: cem__1020.jpg
successfully saved: cem__1021.jpg
successfully saved: cem__1022.jpg
******************failed to save cem__1023.jpg******************************
successfully saved: cem__1024.jpg
******************failed to save cem__1025.jpg******************************
successfully s

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: All-NaN slice encountered
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: All-NaN slice encountered
  """


successfully saved: cem__328.jpg
successfully saved: cem__329.jpg
successfully saved: cem__334.jpg
successfully saved: cem__336.jpg
successfully saved: cem__344.jpg
successfully saved: cem__347.jpg
successfully saved: cem__348.jpg
successfully saved: cem__350.jpg
successfully saved: cem__353.jpg
successfully saved: cem__357.jpg
successfully saved: cem__358.jpg
successfully saved: cem__363.jpg
successfully saved: cem__366.jpg
successfully saved: cem__369.jpg
successfully saved: cem__370.jpg
successfully saved: cem__376.jpg
successfully saved: cem__378.jpg
successfully saved: cem__379.jpg
successfully saved: cem__380.jpg
successfully saved: cem__381.jpg
successfully saved: cem__382.jpg
successfully saved: cem__383.jpg
successfully saved: cem__385.jpg
successfully saved: cem__386.jpg
successfully saved: cem__387.jpg
successfully saved: cem__398.jpg
successfully saved: cem__411.jpg
successfully saved: cem__414.jpg
successfully saved: cem__417.jpg
successfully saved: cem__418.jpg
successful

successfully saved: cem__905.jpg
successfully saved: cem__906.jpg
successfully saved: cem__907.jpg
successfully saved: cem__908.jpg
successfully saved: cem__909.jpg
successfully saved: cem__910.jpg
successfully saved: cem__911.jpg
successfully saved: cem__912.jpg
successfully saved: cem__913.jpg
successfully saved: cem__914.jpg
successfully saved: cem__915.jpg
successfully saved: cem__916.jpg
successfully saved: cem__917.jpg
successfully saved: cem__918.jpg
successfully saved: cem__919.jpg
successfully saved: cem__920.jpg
successfully saved: cem__921.jpg
successfully saved: cem__922.jpg
successfully saved: cem__923.jpg
successfully saved: cem__924.jpg
successfully saved: cem__925.jpg
successfully saved: cem__926.jpg
successfully saved: cem__927.jpg
successfully saved: cem__928.jpg
successfully saved: cem__929.jpg
successfully saved: cem__930.jpg
successfully saved: cem__931.jpg
successfully saved: cem__932.jpg
successfully saved: cem__933.jpg
successfully saved: cem__934.jpg
successful